## Comments Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/SMS YT')

## Install the relevant libraries

In [ ]:
!pip install google-api-python-client
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install demoji
!pip install pandas
!pip install langdetect

In [ ]:
# pip install google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
#  pip install langdetect

In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re   # regular expression

In [ ]:
import pandas as pd

### path to the `client_secret.json` file contains OAuth authorization codes

In [ ]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [ ]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [ ]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=463817865117-itt4a6o2khoeknjgvslu6bbsnr7rpi15.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=4F6VrZ3BvpNpELADOxfF85axoi0HRP&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWhljLzxejIC_ZzGCSetYQsvzhI4GV4h2h3ZukaXtm1lf6cD9tkgiOo


### Set the search query i.e. video of which comments will be extracted

In [ ]:
query = "Web Development"

In [ ]:
query_results = service.search().list(part = 'snippet',q = query,
                                      order = 'relevance', 
                                      type = 'video',
                                      relevanceLanguage = 'en',
                                      safeSearch = 'moderate').execute()

In [ ]:
query_results

### Extract video details i.e. videoID, channel Name, videoTitle, videoDescription

In [ ]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

We only wants to work with the one video so select the first element in list and save it

In [ ]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]


In [ ]:
video_id

'EqzUcMzfV1w'

### Comments Extraction

In [ ]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []


In [ ]:
nextPage_token = None

while 1:
  response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video_id,
                    maxResults = 100, 
                    order = 'relevance', 
                    textFormat = 'plainText',
                    pageToken = nextPage_token
                    ).execute()


  nextPage_token = response.get('nextPageToken')
  for item in response['items']:
      comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
      comment_id_temp.append(item['snippet']['topLevelComment']['id'])
      reply_count_temp.append(item['snippet']['totalReplyCount'])
      like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
      comments_pop.extend(comments_temp)
      comment_id_pop.extend(comment_id_temp)
      reply_count_pop.extend(reply_count_temp)
      like_count_pop.extend(like_count_temp)
        
      video_id_pop.extend([video_id]*len(comments_temp))
      channel_pop.extend([channel]*len(comments_temp))
      video_title_pop.extend([video_title]*len(comments_temp))
      video_desc_pop.extend([video_desc]*len(comments_temp))

  if nextPage_token is  None:
    break

# print(allVideos)

In [ ]:
response

In [ ]:
comments_pop

### As per now comments have been extracted from youtube video and now we will save the comments into pandas dataframe and then write in csv file

In [ ]:


output_dict = {
        'Channel': channel_pop,
        'Video Title': video_title_pop,
        'Video Description': video_desc_pop,
        'Video ID': video_id_pop,
        'Comment': comments_pop,
        'Comment ID': comment_id_pop,
        'Replies': reply_count_pop,
        'Likes': like_count_pop,
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [ ]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183315 entries, 0 to 183314
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Channel            183315 non-null  object
 1   Video Title        183315 non-null  object
 2   Video Description  183315 non-null  object
 3   Video ID           183315 non-null  object
 4   Comment            183315 non-null  object
 5   Comment ID         183315 non-null  object
 6   Replies            183315 non-null  int64 
 7   Likes              183315 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 11.2+ MB


In [ ]:
output_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508


### The above dataframe output, many comments are repeating itself.

Useing `pandas.dataframe.duplicated()` to get the duplicates and `pandas.dataframe.drop_duplicates()` to drop all the duplicates

In [ ]:
duplicates = output_df[output_df.duplicated("Comment ID")]
print("Count of duplicate comments in dataframe", duplicates.shape[0])

Count of duplicate comments in dataframe 182710


In [ ]:
print("count of unique comments in dataframe ",output_df.shape[0]- duplicates.shape[0])

count of unique comments in dataframe  605


In [ ]:
unique_df = output_df.drop_duplicates(subset=['Comment'])

In [ ]:
unique_df.shape

(598, 8)

In the following dataframe all the comments are unique i.e. no comment appears more than once

In [ ]:
unique_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508
5,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380
9,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184
14,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72


## save the dataframe into csv file

In [ ]:
unique_df.to_csv("extraced_comments.csv",index = False)

# Part 2
## Data Cleaning

Install `demoji` library which will be used to remove emojis from the comments

In [ ]:
# pip install demoji

In [ ]:
comments = pd.read_csv('extraced_comments.csv')

In [ ]:
comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72


In [ ]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Channel            598 non-null    object
 1   Video Title        598 non-null    object
 2   Video Description  598 non-null    object
 3   Video ID           598 non-null    object
 4   Comment            598 non-null    object
 5   Comment ID         598 non-null    object
 6   Replies            598 non-null    int64 
 7   Likes              598 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 37.5+ KB


### Remove Emojis 
In this step we will remove emojis from the comments using `demoji`

In [ ]:
import demoji

In [ ]:
comments = pd.read_csv('extraced_comments.csv')

In [ ]:
## this step is necessary as it will download the latest emojis if updaed so as to remove all from the given texts

demoji.download_codes()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  This is separate from the ipykernel package so we can avoid doing imports until


`demoji.findall()` will gives all the emojis present in code with their tags as seen in below<br>
`demoji.replace()` will be useful in replacing emojis from text.

In [ ]:
comments['clean_comments'] = comments['Comment'].apply(lambda x: demoji.replace(x,""))

In [ ]:
comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567,"Hey guys, I know it is a long one, but I wante..."
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508,"Another year, another great yearly guide from ..."
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380,"For programmers, a new year is incomplete wit..."
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184,You have no idea how many times I went through...
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72,Although I'm familiar with alot of the terms h...


Use `langdetect` library to detect language of the comment

In [ ]:
from textblob import TextBlob

In [ ]:
from langdetect import detect

Now add new feature `language` inthe dataframe which signify the language of particular comment

In [ ]:
comments['language'] = 0

In [ ]:
count = 0
for i in range(0,len(comments)):
  temp = comments['clean_comments'].iloc[i]
  count += 1
  try:
    comments['language'].iloc[i] = detect(temp)
  except:
    comments['language'].iloc[i] = "error"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567,"Hey guys, I know it is a long one, but I wante...",en
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508,"Another year, another great yearly guide from ...",en
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380,"For programmers, a new year is incomplete wit...",en
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184,You have no idea how many times I went through...,en
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72,Although I'm familiar with alot of the terms h...,en


 Check for the count of english comments

In [ ]:
comments[comments['language']=='en']['language'].value_counts()

en    541
Name: language, dtype: int64

Save the english comments into csv file

In [ ]:
english_comm = comments[comments['language'] == 'en']

In [ ]:
english_comm.to_csv("english_comments.csv",index = False)

# Part 2.2

### Load the dataset which contain only english comments

In [ ]:
en_comments = pd.read_csv('english_comments.csv',encoding='utf8',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
en_comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567,"Hey guys, I know it is a long one, but I wante...",en
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508,"Another year, another great yearly guide from ...",en
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380,"For programmers, a new year is incomplete wit...",en
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184,You have no idea how many times I went through...,en
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72,Although I'm familiar with alot of the terms h...,en


In [ ]:
en_comments.shape

(541, 10)

### Remove brackets & special characters using Regular Expressions

Now first write the grammar to remove special characters from the comments

In [ ]:
regex = r"[^0-9A-Za-z'\t]"

In [ ]:
copy = en_comments.copy()

In [ ]:
import re

In [ ]:
copy['reg'] = copy['clean_comments'].apply(lambda x:re.findall(regex,x))
copy['regular_comments'] = copy['clean_comments'].apply(lambda x:re.sub(regex,"  ",x))

In [ ]:
copy.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language,reg,regular_comments
0,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Hey guys, I know it is a long one, but I wante...",UgwScEhhjxpfxhw5Bdt4AaABAg,21,567,"Hey guys, I know it is a long one, but I wante...",en,"[ , ,, , , , , , , , ,, , , , , , ...",Hey guys I know it is a long one ...
1,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"Another year, another great yearly guide from ...",UgxGEhGZm1Rf-ssKAhp4AaABAg,2,508,"Another year, another great yearly guide from ...",en,"[ , ,, , , , , , , ., , !]",Another year another great yearly guide...
2,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,"For programmers, a new year is incomplete wit...",UgwumVdLibsBdcZuHY54AaABAg,1,380,"For programmers, a new year is incomplete wit...",en,"[ , ,, , , , , , , , , , , , , , ...",For programmers a new year is incomp...
3,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,You have no idea how many times I went through...,Ugx3SP34sKvs0gfuXmJ4AaABAg,5,184,You have no idea how many times I went through...,en,"[ , , , , , , , , , , , , , ,, , ...",You have no idea how many times I went...
4,Traversy Media,Web Development In 2022 - A Practical Guide,This is my annual guide to take you from start...,EqzUcMzfV1w,Although I'm familiar with alot of the terms h...,UgxFQT5Yg7zieJovgu14AaABAg,0,72,Although I'm familiar with alot of the terms h...,en,"[ , , , , , , , , ,, , , , , , , ...",Although I'm familiar with alot of the ...


## As now we are done with all the preprocessing part of the data.
Now the next step will be sentiment analysis so for that we will make a new dataframe containing only `Video ID`, `Comment ID`,`regular_comments`
<br> after that will write into a new csv file

In [ ]:
dataset = copy[['Video ID','Comment ID','regular_comments']].copy()

In [ ]:
dataset.head()

,Video ID,Comment ID,regular_comments
0,EqzUcMzfV1w,UgwScEhhjxpfxhw5Bdt4AaABAg,Hey guys I know it is a long one ...
1,EqzUcMzfV1w,UgxGEhGZm1Rf-ssKAhp4AaABAg,Another year another great yearly guide...
2,EqzUcMzfV1w,UgwumVdLibsBdcZuHY54AaABAg,For programmers a new year is incomp...
3,EqzUcMzfV1w,Ugx3SP34sKvs0gfuXmJ4AaABAg,You have no idea how many times I went...
4,EqzUcMzfV1w,UgxFQT5Yg7zieJovgu14AaABAg,Although I'm familiar with alot of the ...


In [ ]:
dataset = dataset.rename(columns = {"regular_comments":"comments"})

In [ ]:
# from pycontractions import Contractions

In [ ]:
# dataset['comments'] = dataset['Comment'].apply(lambda x: demoji.replace(x,""))

In [ ]:
dataset.head()

,Video ID,Comment ID,comments
0,EqzUcMzfV1w,UgwScEhhjxpfxhw5Bdt4AaABAg,Hey guys I know it is a long one ...
1,EqzUcMzfV1w,UgxGEhGZm1Rf-ssKAhp4AaABAg,Another year another great yearly guide...
2,EqzUcMzfV1w,UgwumVdLibsBdcZuHY54AaABAg,For programmers a new year is incomp...
3,EqzUcMzfV1w,Ugx3SP34sKvs0gfuXmJ4AaABAg,You have no idea how many times I went...
4,EqzUcMzfV1w,UgxFQT5Yg7zieJovgu14AaABAg,Although I'm familiar with alot of the ...


In [ ]:
dataset.to_csv("Dataset.csv",index = False)